In [1]:
%reset -fs

import pandas as pd
import numpy as np
from sklearn import metrics

# pd.set_option('max_columns', 100)
pd.set_option("display.max_colwidth", 1)

In [2]:
df = pd.read_csv('tweets_01-08-2021.csv', encoding='utf-8')

In [3]:
len(df)

56571

In [4]:
df.head(30)

,id,text,isRetweet,isDeleted,device,favorites,retweets,date,isFlagged
0,98454970654916608,Republicans and Democrats have both created our economic problems.,f,f,TweetDeck,49,255,2011-08-02 18:07:48,f
1,1234653427789070336,"I was thrilled to be back in the Great city of Charlotte, North Carolina with thousands of hardworking American Patriots who love our Country, cherish our values, respect our laws, and always put AMERICA FIRST! Thank you for a wonderful evening!! #KAG2020 https://t.co/dNJZfRsl9y",f,f,Twitter for iPhone,73748,17404,2020-03-03 01:34:50,f
2,1218010753434820614,RT @CBS_Herridge: READ: Letter to surveillance court obtained by CBS News questions where there will be further disciplinary action and cho…,t,f,Twitter for iPhone,0,7396,2020-01-17 03:22:47,f
3,1304875170860015617,"The Unsolicited Mail In Ballot Scam is a major threat to our Democracy, &amp; the Democrats know it. Almost all recent elections using this system, even though much smaller &amp; with far fewer Ballots to count, have ended up being a disaster. Large numbers of missing Ballots &amp; Fraud!",f,f,Twitter for iPhone,80527,23502,2020-09-12 20:10:58,f
4,1218159531554897920,RT @MZHemingway: Very friendly telling of events here about Comey's apparent leaking to compliant media. If you read those articles and tho…,t,f,Twitter for iPhone,0,9081,2020-01-17 13:13:59,f
5,1217962723234983937,RT @WhiteHouse: President @realDonaldTrump announced historic steps to protect the Constitutional right to pray in public schools! https://…,t,f,Twitter for iPhone,0,25048,2020-01-17 00:11:56,f
6,1223640662689689602,Getting a little exercise this morning! https://t.co/fyAAcbhbgk,f,f,Twitter for iPhone,285863,30209,2020-02-01 16:14:02,f
7,1319501865625784320,https://t.co/4qwCKQOiOw,f,f,Twitter for iPhone,130822,19127,2020-10-23 04:52:14,f
8,1319500520126664705,https://t.co/VlEu8yyovv,f,f,Twitter for iPhone,153446,20275,2020-10-23 04:46:53,f
9,1319500501269041154,https://t.co/z5CRqHO8vg,f,f,Twitter for iPhone,102150,14815,2020-10-23 04:46:49,f


In [5]:
df.groupby('isFlagged').count()

,id,text,isRetweet,isDeleted,device,favorites,retweets,date
isFlagged,,,,,,,,
f,56267,56267,56267,56267,56267,56267,56267,56267
t,304,304,304,304,304,304,304,304


In [6]:
# Text preprocessing steps - remove numbers, URL and punctuation
import re
import string

alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
punc = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ',x)
no_url = lambda x: re.sub(r'http\S+', '', x)

df['text'] = df.text.map(alphanumeric).map(no_url).map(punc)
df.head(50)

,id,text,isRetweet,isDeleted,device,favorites,retweets,date,isFlagged
0,98454970654916608,Republicans and Democrats have both created our economic problems,f,f,TweetDeck,49,255,2011-08-02 18:07:48,f
1,1234653427789070336,I was thrilled to be back in the Great city of Charlotte North Carolina with thousands of hardworking American Patriots who love our Country cherish our values respect our laws and always put AMERICA FIRST Thank you for a wonderful evening,f,f,Twitter for iPhone,73748,17404,2020-03-03 01:34:50,f
2,1218010753434820614,RT CBS Herridge READ Letter to surveillance court obtained by CBS News questions where there will be further disciplinary action and cho…,t,f,Twitter for iPhone,0,7396,2020-01-17 03:22:47,f
3,1304875170860015617,The Unsolicited Mail In Ballot Scam is a major threat to our Democracy amp the Democrats know it Almost all recent elections using this system even though much smaller amp with far fewer Ballots to count have ended up being a disaster Large numbers of missing Ballots amp Fraud,f,f,Twitter for iPhone,80527,23502,2020-09-12 20:10:58,f
4,1218159531554897920,RT MZHemingway Very friendly telling of events here about Comey s apparent leaking to compliant media If you read those articles and tho…,t,f,Twitter for iPhone,0,9081,2020-01-17 13:13:59,f
5,1217962723234983937,RT WhiteHouse President realDonaldTrump announced historic steps to protect the Constitutional right to pray in public schools,t,f,Twitter for iPhone,0,25048,2020-01-17 00:11:56,f
6,1223640662689689602,Getting a little exercise this morning,f,f,Twitter for iPhone,285863,30209,2020-02-01 16:14:02,f
7,1319501865625784320,,f,f,Twitter for iPhone,130822,19127,2020-10-23 04:52:14,f
8,1319500520126664705,,f,f,Twitter for iPhone,153446,20275,2020-10-23 04:46:53,f
9,1319500501269041154,,f,f,Twitter for iPhone,102150,14815,2020-10-23 04:46:49,f


In [7]:
df['text'] = df['text'].apply(lambda x: x.strip()).replace('', np.nan) # fill empty cells w/ Null values

In [8]:
df.head(50)

,id,text,isRetweet,isDeleted,device,favorites,retweets,date,isFlagged
0,98454970654916608,Republicans and Democrats have both created our economic problems,f,f,TweetDeck,49,255,2011-08-02 18:07:48,f
1,1234653427789070336,I was thrilled to be back in the Great city of Charlotte North Carolina with thousands of hardworking American Patriots who love our Country cherish our values respect our laws and always put AMERICA FIRST Thank you for a wonderful evening,f,f,Twitter for iPhone,73748,17404,2020-03-03 01:34:50,f
2,1218010753434820614,RT CBS Herridge READ Letter to surveillance court obtained by CBS News questions where there will be further disciplinary action and cho…,t,f,Twitter for iPhone,0,7396,2020-01-17 03:22:47,f
3,1304875170860015617,The Unsolicited Mail In Ballot Scam is a major threat to our Democracy amp the Democrats know it Almost all recent elections using this system even though much smaller amp with far fewer Ballots to count have ended up being a disaster Large numbers of missing Ballots amp Fraud,f,f,Twitter for iPhone,80527,23502,2020-09-12 20:10:58,f
4,1218159531554897920,RT MZHemingway Very friendly telling of events here about Comey s apparent leaking to compliant media If you read those articles and tho…,t,f,Twitter for iPhone,0,9081,2020-01-17 13:13:59,f
5,1217962723234983937,RT WhiteHouse President realDonaldTrump announced historic steps to protect the Constitutional right to pray in public schools,t,f,Twitter for iPhone,0,25048,2020-01-17 00:11:56,f
6,1223640662689689602,Getting a little exercise this morning,f,f,Twitter for iPhone,285863,30209,2020-02-01 16:14:02,f
7,1319501865625784320,NaN,f,f,Twitter for iPhone,130822,19127,2020-10-23 04:52:14,f
8,1319500520126664705,NaN,f,f,Twitter for iPhone,153446,20275,2020-10-23 04:46:53,f
9,1319500501269041154,NaN,f,f,Twitter for iPhone,102150,14815,2020-10-23 04:46:49,f


In [9]:
df = df.dropna() #drop null rows
df.head(50)

,id,text,isRetweet,isDeleted,device,favorites,retweets,date,isFlagged
0,98454970654916608,Republicans and Democrats have both created our economic problems,f,f,TweetDeck,49,255,2011-08-02 18:07:48,f
1,1234653427789070336,I was thrilled to be back in the Great city of Charlotte North Carolina with thousands of hardworking American Patriots who love our Country cherish our values respect our laws and always put AMERICA FIRST Thank you for a wonderful evening,f,f,Twitter for iPhone,73748,17404,2020-03-03 01:34:50,f
2,1218010753434820614,RT CBS Herridge READ Letter to surveillance court obtained by CBS News questions where there will be further disciplinary action and cho…,t,f,Twitter for iPhone,0,7396,2020-01-17 03:22:47,f
3,1304875170860015617,The Unsolicited Mail In Ballot Scam is a major threat to our Democracy amp the Democrats know it Almost all recent elections using this system even though much smaller amp with far fewer Ballots to count have ended up being a disaster Large numbers of missing Ballots amp Fraud,f,f,Twitter for iPhone,80527,23502,2020-09-12 20:10:58,f
4,1218159531554897920,RT MZHemingway Very friendly telling of events here about Comey s apparent leaking to compliant media If you read those articles and tho…,t,f,Twitter for iPhone,0,9081,2020-01-17 13:13:59,f
5,1217962723234983937,RT WhiteHouse President realDonaldTrump announced historic steps to protect the Constitutional right to pray in public schools,t,f,Twitter for iPhone,0,25048,2020-01-17 00:11:56,f
6,1223640662689689602,Getting a little exercise this morning,f,f,Twitter for iPhone,285863,30209,2020-02-01 16:14:02,f
11,1319496349092511744,RT EricTrump,t,f,Twitter for iPhone,0,8921,2020-10-23 04:30:19,f
12,1236502342121541632,RT GOPChairwoman The economic boom continues \n\n jobs added in February \n\n jobs added since realDonaldTrump was elected\n\nUnemploym…,t,f,Twitter for iPhone,0,8681,2020-03-08 04:01:46,f
13,1225835449379258368,RT TomFitton Vindman s behavior is a scandal He should be removed from the RealDonaldTrump White House ASAP to protect our foreign poli…,t,f,Twitter for iPhone,0,7679,2020-02-07 17:35:20,f


In [18]:
from datetime import datetime as dt

In [20]:
df["date"] = pd.to_datetime(df.date , format="%Y/%m/%d %H:%M:%S") #create date_time column

In [27]:
df_before = df[(df['date'] <= "2017-01-19")]
df_before

,id,text,isRetweet,isDeleted,device,favorites,retweets,date,isFlagged
0,98454970654916608,Republicans and Democrats have both created our economic problems,f,f,TweetDeck,49,255,2011-08-02 18:07:48,f
1601,336155055575150594,Yes,f,f,Twitter for Android,0,1,2013-05-19 16:23:05,f
1615,274563166384582657,Thanks you will love it,f,f,Twitter Web Client,1,4,2012-11-30 17:18:54,f
1620,274562988646727681,CelebApprentice bretmichaels You won t be disappointed,f,f,Twitter Web Client,0,3,2012-11-30 17:18:12,f
1696,284759862976581632,WandaSalley Thanks have fun,f,f,Twitter Web Client,1,2,2012-12-28 20:36:56,f
...,...,...,...,...,...,...,...,...,...
45874,815449868739211265,RT DonaldJTrumpJr Happy new year everyone newyear family vacation familytime,t,f,Twitter for iPhone,0,5548,2017-01-01 06:49:33,f
45875,815433444591304704,RT EricTrump was such an incredible year for our entire family My beautiful wife LaraLeaTrump made it even better 🇺🇸🇺🇸,t,f,Twitter for iPhone,0,5601,2017-01-01 05:44:17,f
45876,815433217595547648,RT Reince Happy New Year God s blessings to you all Looking forward to incredible things in realDonaldTrump will Make America,t,f,Twitter for iPhone,0,5811,2017-01-01 05:43:23,f
45877,815432169464197121,RT DanScavino On behalf of our next POTUS amp TeamTrump HappyNewYear AMERICA🇺🇸 t co,t,f,Twitter for iPhone,0,4562,2017-01-01 05:39:13,f


In [28]:
df_after = df[(df['date'] > "2017-01-20")]
df_after

,id,text,isRetweet,isDeleted,device,favorites,retweets,date,isFlagged
1,1234653427789070336,I was thrilled to be back in the Great city of Charlotte North Carolina with thousands of hardworking American Patriots who love our Country cherish our values respect our laws and always put AMERICA FIRST Thank you for a wonderful evening,f,f,Twitter for iPhone,73748,17404,2020-03-03 01:34:50,f
2,1218010753434820614,RT CBS Herridge READ Letter to surveillance court obtained by CBS News questions where there will be further disciplinary action and cho…,t,f,Twitter for iPhone,0,7396,2020-01-17 03:22:47,f
3,1304875170860015617,The Unsolicited Mail In Ballot Scam is a major threat to our Democracy amp the Democrats know it Almost all recent elections using this system even though much smaller amp with far fewer Ballots to count have ended up being a disaster Large numbers of missing Ballots amp Fraud,f,f,Twitter for iPhone,80527,23502,2020-09-12 20:10:58,f
4,1218159531554897920,RT MZHemingway Very friendly telling of events here about Comey s apparent leaking to compliant media If you read those articles and tho…,t,f,Twitter for iPhone,0,9081,2020-01-17 13:13:59,f
5,1217962723234983937,RT WhiteHouse President realDonaldTrump announced historic steps to protect the Constitutional right to pray in public schools,t,f,Twitter for iPhone,0,25048,2020-01-17 00:11:56,f
...,...,...,...,...,...,...,...,...,...
56566,1319485303363571714,RT RandPaul I don’t know why JoeBiden thinks he can continue to lie about this \n\nHe wants to ban fracking and end all fossil fuels like…,t,f,Twitter for iPhone,0,20683,2020-10-23 03:46:25,f
56567,1319484210101379072,RT EliseStefanik President realDonaldTrump excels at communicating directly to the American people \n\nJoe Biden communicates to the DC B…,t,f,Twitter for iPhone,0,9869,2020-10-23 03:42:05,f
56568,1319444420861829121,RT TeamTrump LIVE Presidential Debate \n\nText VOTE to,t,f,Twitter for iPhone,0,8197,2020-10-23 01:03:58,f
56569,1319384118849949702,Just signed an order to support the workers of Delphi Corporation and make sure that we protect the pensions of all American workers Obama Biden FAILED American workers and FAILED the workers of Delphi I ALWAYS put American workers FIRST,f,f,Twitter for iPhone,176289,36001,2020-10-22 21:04:21,f
